### 작업전(t일 이전) 할 거
0. 조회 > 변제오류까지 작업 > 변제오류건 최신화 누른 후 다운로드 다시

1. 항고건 살피기 : 아무나 한 건 검색, 아무셀이나 필터적용해서 CUSTOMIZE단추 열기, 필터함수 불러오고 원래 검색조건은 삭제 후 검색(D:\3.자산\전산 dataset)
  - 건수 3건 이상이면 업데이트는 엑셀로 하는 게 편한듯.
2. 12월에 일반회생(연납건) 입금회차 확인
[20420482 보증인]
[20420209 차주]
[n1027937 : 20426827,20426904 차주]
입금월 12, 12, 12, 12

- 수정대상건에 대해 전산수정작업 마친 후 t+1일에 다운로드()
1. 우클릭 후 '1.채권자목록,일반내용,진행내용 표시'선택할 것!!

- 기준일 개회 입금 t+1에 다운로드. 계좌별 표시 x !!!!!!!!!!!!!!!!!!!!!!!!!!

### import 및 기본변수

In [81]:

import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")
from datetime import datetime
from datetime import timedelta
import calendar
from dateutil.relativedelta import relativedelta
from openpyxl import Workbook
from openpyxl.styles import Font


# 파일 읽기
path_base = r"D:\3.자산\전산 dataset"
########################################################
company = "솔림"
cutoff = "240704" # 폴더명
name_dataset = "개인회생새창_20240705_0929" 
ext = ".xlsx"   ####--------

# previous_cutoff = "240108" # 폴더명
# previous_name_dataset = "개인회생새창_20240109_0943_기준데이터" # 기준데이터 계좌키 미분리
########################################################

검색용사명 = "(솔림|헬프)" if company=="솔림" else "대성인베스트"

cutoff_dt = datetime.strptime(cutoff, "%y%m%d")
조회일_dt = datetime.strptime(name_dataset.split("_")[1], "%Y%m%d")
조회일 = datetime.strftime(조회일_dt, "%Y-%m-%d")

path_wd = join(path_base, company, cutoff)
# path_previous = join(path_base, company, previous_cutoff, previous_name_dataset+".xlsx") # 전체시트 읽어야 하므로 pkl 아님
path_result = join(path_wd, name_dataset+"_기준데이터.xlsx")

rehabilitation_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '사건키':str, '신고계좌':str, '입금계좌':str}
grt_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}

# 금지명령(중지명령) : 전산은 금지명령이 기각되는 경우를 무시하고 신청서제출만으로 금지로 처리함.
# 접수 : 금지명령신청만 한 경우나, 기각된 경우를 따로 표시하기 위해 추가함
항고인용사건_현재결과종류 = {"개시":"개인회생절차개시결정\x7f", "금지": r"금지명령\(|중지명령\x7f", # r"(금지|중지)명령신청서? 제출\x7f", 
          "면책허가결정":"면책허가결정공고\x7f", "신청취하" : "종국 : 신청취하\x7f", "이송" : "종국 : 이송\x7f", "인가" : r"(회생|변제)계획인가결정\x7f","인용" : "종국 : 인용\x7f",
          "폐지취소결정" :"폐지취소결정\x7f", "폐지취소결정송달" : "폐지취소결정 송달\x7f"}

기각폐지항고 = {"기각":"종국 : 기각\x7f", "기각결정":"개인회생절차개시신청 기각결정\x7f", 
          "폐지결정" : "폐지결정\(", "폐지":"종국 : 폐지\x7f", 
          "항고" : "항고장"}

현재결과종류 = {key: value for d in [항고인용사건_현재결과종류, 기각폐지항고] for key, value in d.items()}

def 현재결과찾기(진행내용:str) -> str:
    # 진행내용 최근 결정내용을 찾아 현재결과 리턴
    for value in reversed(진행내용.split("\n")) :
        for i, v in 현재결과종류.items() :
            if re.search(v, value) :
                return i
            
    return "접수" # 처음까지 왔는데 아무런 결정내용도 확인할 수 없다면

def 항고사건_현재결과찾기(진행내용:str) -> str :
    for value in reversed(진행내용.split("\n")) :
        for i, v in 항고인용사건_현재결과종류.items() :
            if re.search(v, value) :
                return i
    return "접수" # 처음까지 왔는데 아무런 결정내용도 확인할 수 없다면

def is_last_day_of_month(date_str):
    # 문자열을 datetime 객체로 변환
    date = datetime.strptime(date_str, "%Y-%m-%d")
    
    # 해당 월의 마지막 날 계산
    last_day = calendar.monthrange(date.year, date.month)[1]
    
    # 입력 날짜가 그 달의 마지막 날인지 확인
    return date.day == last_day

def 현재회차정보찾기(row) :
    """ 변제구간값들을 받아서 return 당사변제시작일, 실변제일, 실변제금, 말일여부 """
    # 조회일전일이 말인인 경우만 유의하면 됨. 예컨대 start이 1/31인데 
    # 조회일이 3/1이면 +1되는 게 맞고, 2/28이면 
    # -1, -2:분납후? 어쨌든 구간 마지막에 나옴
    
    당사변제시작일, 실변제일, 실변제금, 말일여부, 구간번호 = "", "", "", "", 0 # 기본값 : False
    
    # 값 없음[시작] ---------------------------------------------------
    # 변제금합계가 없는 경우(총분납금은 제외채권이라도 이미 읽어졌을 수도 있음)
    # 제외채권처리 했다면 사실 이 경우는 없는 경우
    if row["변제금합계"] == 0 :
        return 당사변제시작일, 실변제일, 실변제금, 말일여부
    
    # 변제조회 실패, 조회성공했다면 기간1에는 무조건 값이 있다.
    if row['변제기간1'] == "" : 
        return 당사변제시작일, 실변제일, 실변제금, 말일여부
    # 값 없음[끝] ---------------------------------------------------
    
    # 변제조회 성공
    else : 
        # 구간순회하며 현재 회차에 맞는 정보찾기
        현재회차 = int(row['현재회차'])
        
        # 1. 먼저 당사변제시작일 찾기. 반복분 따로 돌려야 오류 없겠다.
        for i in range(1, 11):
            if row[f'변제예정액{i}'] > 0 :
                당사변제시작일 = row[f'변제기간{i}'][:10]
                break
        
        # 당사 변제일 미도래
        if datetime.strptime(당사변제시작일,"%Y-%m-%d") > cutoff_dt :
            # 240705 현재 인가&변제조회 정상건 중 변제기일,구간별납입일자,실변제일이 서로 다른 경우는 말일인 경우밖에 없없음.
            # 결국 실변제일은 안 구해도 됐음...일단 혹시 모르니 그냥 두고, 변제기일을 출력하자.
            실변제일 = row["변제기일"] 
            if len(실변제일) == 2 : # 전산도 에러 아니면, 이 조건 달아야 아래 if문 오류 안날테니
                if is_last_day_of_month(row["신고조회일"][:8]+실변제일) : # 변제조회일(월) 기준으로 전산이 변제기일 잡을 것이므로
                    말일여부 = "말일"
            return 당사변제시작일, 실변제일, 실변제금, 말일여부
        
        # 현재회차 기준 전체 변제 종료
        if datetime.strptime(row["변제종료일"],"%Y-%m-%d") <= cutoff_dt :
            실변제일 = row["변제기일"] 
            if len(실변제일) == 2 : # 전산도 에러 아니면, 이 조건 달아야 아래 if문 오류 안날테니
                if is_last_day_of_month(row["신고조회일"][:8]+실변제일) : # 변제조회일(월) 기준으로 전산이 변제기일 잡을 것이므로
                    말일여부 = "말일"
            return 당사변제시작일, 실변제일, 실변제금, 말일여부
        
        # 2. 다음으로 실변제일,실변제금,말일여부 찾기
        for i in range(1, 11):
            
            회차값 = row[f'변제회차{i}']
            
            # 회차값 없으면 다음 구간으로
            if 회차값 == "":
                continue
            
            # 현재회차에 맞는 구간번호 찾기
            if '~' in str(회차값):
                start, end = map(int, 회차값.split('~'))
                if start <= 현재회차 <= end:
                    구간번호 = i
            else:
                if int(회차값) == 현재회차:
                    구간번호 = i
                    
            # 맞는 구간번호 찾았으면 값 확정하고 리턴
            if 구간번호 : # != 0
                실변제금 = row[f'변제예정액{구간번호}']
                현재회차변제시작일 = row[f'변제기간{구간번호}'][:10]
                실변제일 = 현재회차변제시작일[-2:] 
                
                #실변제일(변제시작일의 날짜값)이 말일인지 체크
                if is_last_day_of_month(현재회차변제시작일) :
                    말일여부 = "말일"
                
                return 당사변제시작일, 실변제일, 실변제금, 말일여부
            else : # == 0
                continue # 반복문 계속
            
        # 반복문 끝나도 뭔가가 잘못됐다면 예측못한 오류
        return "오류", "오류", "오류", "오류"
                
def 최근변제예정일찾기(row, reference_date):
    """ reference_date는 날짜거나 %Y-%m-%d형태의 문자열일것"""
    try : 
        실변제일 = int(row['변제기일']) # 실변제일에서 변제기일로 바꿈
    except ValueError: # 실변제일 없는 경우
        return ""
    if isinstance(reference_date, str) :
        reference_date = datetime.strptime(reference_date,'%Y-%m-%d')
    조회년도 = reference_date.year
    조회월 = reference_date.month
    
    # 참조일 : 현재월의 1일(1일은 어느 달이나 있으니까)
    참조일 = datetime(조회년도, 조회월, 1)
    참조일이전달 = 참조일 - relativedelta(months=1)
    참조일다음달 = 참조일 + relativedelta(months=1)
    
    # 실변제일 대입하기
    def get_payment_date(치환할날, 실변제일):
        마지막날 = calendar.monthrange(치환할날.year, 치환할날.month)[1]
        if 실변제일 > 마지막날:
            return datetime(치환할날.year, 치환할날.month, 마지막날)
        else:
            return datetime(치환할날.year, 치환할날.month, 실변제일)
    
    # 이번달, 지난달, 다음달 변제일 계산
    이번달변제일 = get_payment_date(참조일, 실변제일)
    이전달변제일 = get_payment_date(참조일이전달, 실변제일)
    다음달변제일 = get_payment_date(참조일다음달, 실변제일)
  
    
    # 조회일과 가장 가까운 날짜 계산
    최근변제예정일 = min([이번달변제일, 이전달변제일, 다음달변제일], 
                       key=lambda date: abs(참조일 - date))
    
    return 최근변제예정일.strftime('%Y-%m-%d')

def save_df_to_excel(df, filename, font_size=9):
    # ExcelWriter를 openpyxl 엔진과 함께 사용
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Sheet1')
        
        # 워크북과 워크시트 객체 가져오기
        workbook = writer.book
        sheet = writer.sheets['Sheet1']
        
        # 글씨 크기 변경 (폰트 설정)
        font = Font(size=font_size)
        
        # 모든 셀에 대해 글씨 크기 설정
        for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
            for cell in row:
                cell.font = font

# 채권상태분류
comp_closing = {
    # 채권 자체의 소멸
    "종료" : r"(?<!청산)종결(?!요청)|매각(?!예정)|(?<!피)환매(?!예정)|DS|완납|완제|종료",
    # 비정상-종결요청
    "종결요청" : r"종결요청|매각\(예정\)|(?<!피)환매\(예정\)|패소",
    # 시효
    "시효" : r"시효|완성|만료", # 부활 제외함 231222
    # 채무자별로 체크
    "면책" : r"면책|면탈",  
    "불가" : r"추심제외|상각|포기|파산(?!(\(기각\)|\s누락))|사망(?!\(상속승인\))|폐업|청산종결|패소", # 상각, 제외, 포기는 이미 보증인까지 고려한 결과일 수 있음 # 담당자 누락제외는 채무상태에 적용하면 안 됨.(계좌의 메모를 봐야함)
    "공채권" : r"공채권"
    # 이민, 말소, 실종은 정상으로
    }

종료 = re.compile(comp_closing["종료"])
종결요청 = re.compile(comp_closing["종결요청"])
시효 = re.compile(comp_closing["시효"])
면책 = re.compile(comp_closing["면책"])
불가 = re.compile(comp_closing["불가"])
공채권 = re.compile(comp_closing["공채권"])

# 칼럼정의
기본조회칼럼 = ["채무자키","계좌키","분납키","분납자관계","분납자성명인","주민번호인","관할법원","사건번호", "인가미인가", "현재결과", "분납메모","분납비고"]
회차칼럼 = ["총분납회차","현재회차", "잔여회차", "납입회차", "미납회차", "초과입금"]
법원회차칼럼 = ["전체변제회차","현재변제회차","잔여변제회차","채무자미납회차","매월변제예정금액","출금대상잔액"]
차이칼럼 = ["현재회차차이", "미납회차차이", "출금대상회차"]
변제조회칼럼 = ["변제현황조회","신고조회일","신고조회결과"]
은행칼럼 = ["신고은행","신고계좌","신고채권자","입금은행", "입금계좌", "입금채권자"]
채권번호칼럼 = ["변제채권번호입력", "변제채권번호스캔", "변제채권번호"]
회차정보칼럼 = ["최근변제예정일","당사변제시작일","변제시작일","현재회차변제일", "현재회차변제금", "말일여부", "변제기일"]
회차구간칼럼 = ["변제회차1", "변제기간1", "변제회차2", "변제기간2", "변제회차3", "변제기간3", "회차수정오류여부"]
임시칼럼 = ["상태및담당자"]

### 파일 읽기 및 저장

In [99]:
# previous = pd.read_excel(path_previous, sheet_name="개인회생(전체)", dtype={'채무자키':str, '분납키':str, '계좌키':str})
if ext == ".xlsx" : 
    path_raw_data = join(path_wd, name_dataset+ext)
    raw_data_ori = pd.read_excel(path_raw_data, dtype=rehabilitation_dtype).fillna("")

    if raw_data_ori.loc[0,"진행내용"] == "" :
        print("개인회생 엑셀다운로드시 우클릭후 '1.채권자목록,일반내용,진행내용 표시'를 선택해야 합니다.")
    else : 
        pass
    
    def format_date(value):
        # 값이 빈 문자열이거나 '일'만 있는 경우 처리
        if value == '' or value == '일':
            return ''
        
        # 숫자만 추출
        day = ''.join(filter(str.isdigit, value))
        
        # 두 자리 문자열로 변환
        return day.zfill(2)

    # '변제기일' 열에 함수 적용 : '일'떼고 두자리 문자열로 저장
    raw_data_ori['변제기일'] = raw_data_ori['변제기일'].apply(format_date)

    # 법원오타 정정해주기
    if company == "솔림":
        condition = (raw_data_ori['분납키'] == "20106495") & (raw_data_ori['현재결과'] == "인가")
        idx = raw_data_ori[condition].index
        raw_data_ori.loc[idx, '변제기간2'] = "2022-06-07~2022-06-07"
    else :
        print("법원에서 변제기간 오타냈던 거(20409923) 이제 면책됐다. 이 코드 삭제해도 되게다.")
    
    # 사건번호 정렬
    raw_data_ori.sort_values(by=["관할법원", "사건번호"], inplace=True)
                
    # pkl 저장
    if not os.path.exists(join(path_wd, "pkl")) : 
        os.mkdir(join(path_wd, "pkl"))
    raw_data_ori.to_pickle(join(path_wd, "pkl", name_dataset+".pkl"))

# pkl 읽기    
else : 
    path_raw_data = join(path_wd, "pkl", name_dataset+ext)
    # path_raw_data = join(path_wd, "pkl", name_dataset+"_진행중_계좌키분리_기준데이터"+ext)
    raw_data_ori = pd.read_pickle(path_raw_data)

### 재시작점-----------

In [ ]:
raw_data = raw_data_ori.copy()

### 최근 접수건만 남기기

In [56]:
# 접수일 없는 건은 모두 중단건이다.(자동으로 불러오지 못한 거니까). 따라서 일단 제일 뒤로 보내기 위해 임의의 날짜값으로 대체
raw_data.loc[raw_data.query('접수일==""').index,"접수일"] = "1975-01-01"
# 역정렬
raw_data = raw_data.sort_values(by='접수일', ascending=False)

# 삭제할 것 중 분납상태 진행 있는지 체크
삭제할중복값 = raw_data[raw_data.duplicated(subset=['계좌키', '분납자성명인'], keep='first')]
if len(삭제할중복값.query('분납상태 == "진행"')) : 
    display(삭제할중복값)
else : 
    print(len(raw_data))
    raw_data.drop_duplicates(subset=['계좌키', '분납자성명인'], keep='first', inplace=True)
    raw_data.reset_index(drop=True, inplace=True)
    print(len(raw_data))

9670
9259


### 사전점검 : 동일 사건 중 현재결과 및 분납상태 다른 거 있는지 

In [57]:
동일사건 = raw_data.groupby(by=['관할법원', '사건번호'])
상이한현재결과 = 동일사건.filter(lambda x: x['현재결과'].nunique() != 1)
상이한현재결과.query('분납상태!="중단"').sort_values(["사건번호"])[["채무자키","분납키","주민번호인","분납자관계","분납자성명인","관할법원","사건번호","현재결과","분납상태"]]
# 분납상태가 모두 중단이면 현재결과 달라도 상관없음

,채무자키,분납키,주민번호인,분납자관계,분납자성명인,관할법원,사건번호,현재결과,분납상태


### 인가미인가 작성

In [58]:
# 컬럼 순서 바꾸기, 인트를 문자열로(타입때메 종료제거보다 나중에 나와야)
raw_data.insert(1, "분납키", raw_data.pop("분납키"))
raw_data.insert(7, "현재결과", raw_data.pop("현재결과"))
raw_data.insert(8, "분납상태", raw_data.pop("분납상태"))
raw_data.insert(9, "검수상태", raw_data.pop("검수상태"))

# 현재결과 종류
# 인가 : 인가
# 미인가 : 개시, 금지, 금지명령
# 폐지 : 절차폐지결정일, 폐지결정, 기각, 기각결정, 신청취하, 폐지
# 면책 : 인용, 면책허가결정, 면책,              * 종결은 면책목록에서 제외함
# (230131) 개편
# 변제계획인가일 확인 할 것 : 폐지취소결정송달, 폐지취소결정
# 분납메모, 변제계획인가일 확인 할 것

# 인가미인가
인가미인가 = []
for i, v in tqdm(raw_data[["현재결과", "분납키", "분납메모", "변제계획인가일", "항고", "진행내용"]].iterrows(), total=len(raw_data.index)) :
    result, opb = "", 0
    
    결과 = v.현재결과
    # 현재결과에 따라 인가,미인가,폐지,면책,이송,체크 정하기
    # 인가
    if 결과 == "인가" or (결과 =="종결" and re.search("일반회생", v.분납메모)): # 종결은 수기등록 하는 거 아니면 회단사건에서만 확인되고 있다. 
        result = "인가"                 # 분납메모가 있으면 회단사건의 종결은 인가와 다름없다.(법원 절차만 종결됐다는 소리이므로). 분납메모가 없다면 체크로 빠지도록 한다. 완제됐다면 분납상태에서 걸리겄지.
        
    # 미인가
    elif re.search("개시|금지|접수", 결과) : 
        result = "미인가"
        
    # 폐지
    elif (re.search("폐지결정|기각|취하", 결과)) or (결과 == "폐지") : 
        result = "폐지"
        
    # 폐지취소 - 분납메모 미사용
    elif re.search("폐지취소", 결과) : 
        result = "미인가" if v.변제계획인가일 == "" else "인가"
            
    # 면책
    elif re.search("인용|면책|완제", 결과) : # 일부 채권사것만 완제하는 경우가 있다. 종결로 적지 않고 완제로 적을 것!!!
        result = "면책"
        
    # 이송 : 분납상태 중지 & 새사건 입력되었어야
    elif re.search("이송", 결과) :
        result = "이송"
        
    # 실사필요 : 항고인데 현재결과가 어디에도 해당하지 않을 때 or 새로운 현재결과가 나온 경우
    elif re.search("항고", 결과) : 
        항고사건결과 = v.분납메모.split(",")[0]
        채무자항고 = False if re.search("채권", v.항고) else True
        
        # 항고사건결과 아직 없음 : 체크
        if 항고사건결과 == "" : 
            result = "체크_항고사건결과없음"
            
        # 미결정 확인 완료 : 인가일 있는지에 따라
        elif 항고사건결과 == "미결정" : 
            result = "미인가" if v.변제계획인가일 == "" else "인가"
            
        # 항고 인용, 기각취소, 폐지취소 ----------------ㅁㅁㅁ 현재 항고사건없이 바로 결정나온것에 기각취소, 기각, 폐지취소는 확인되었으나 폐지(?!취소)는 확인이 안 되어 코드에 미반영상태
        elif re.search("환송|인용|기각취소|폐지취소", 항고사건결과) :
            if 채무자항고 :
                result = "미인가" if v.변제계획인가일 == "" else "인가"
                raw_data.loc[i, "현재결과"] = 항고사건_현재결과찾기(v.진행내용)
            else : 
                result = "체크_채권자항고" # 채권자 항고에 대한 기각취소는 없어야하고, 파기환송과 인용은 아직 사례를 못 봐서 데이터 수집 필요
        # 취하, 기각, 각하
        elif re.search("취하|각하|기각(?!취소)", 항고사건결과) : # ----------------ㅁㅁㅁ 항고사건없이 항고장제출 후 폐지가 나오는 경우 있다면 추가해야
            if 채무자항고 : 
                result = "폐지"
            else : 
                result = "미인가" if v.변제계획인가일 == "" else "인가"
                raw_data.loc[i, "현재결과"] = 항고사건_현재결과찾기(v.진행내용)
        # 분납메모에 엉뚱한 말이 적힌 경우
        else : 
            result = "체크_항고사건결과를 잘못적음"
    
    # 위의 어떤 경우에도 해당하지 않는 경우 : 없어야 함
    else : result = "체크_항목에 없는 현재결과"

    인가미인가.append(result)

raw_data.insert(7, "인가미인가", 인가미인가)
raw_data.insert(11, "반영OPB", None)

# 폐지취소 이전 파일에서 불러왔던 코드
# elif re.search("폐지취소", 결과) :  # 폐지취소결정송달 포함
#     temp = previous.query('분납키==@v.분납키')["인가미인가"].values
#     if len(temp) == 1 : result = temp[0]
#     else : result = "체크"

100%|██████████| 9259/9259 [00:00<00:00, 46922.42it/s]


### 본격 오류 체크
- 체크 순서 지켜야
- 오류관련 메시지는 분납비고에 작성
- 멘트기본형식 : [채권번호|회차|..]+오류아님

#### 1.진행점검 : (개시) 채권자, 계좌신고여부
- 분납비고 prefix : 신고_

In [48]:
raw_data["상태및담당자"] = raw_data["채무상태"] + raw_data["담당자"]
분납비고prefix = "신고_"

### 진행건 중 채권자신고, 계좌신고 안 된 것 : 인가/미인가/not종결/메모는무시/개시결정
미종결개시건 = raw_data.query(
                        '분납상태 == "진행" and 인가미인가.str.contains("인가") and 현재결과!="종결" \
                        and ~(상태및담당자.str.contains(@종료) or 상태및담당자.str.contains(@종결요청)) \
                        and 개시결정일 != ""'
                        )

미종결개시건[["채권자신고점검", "계좌신고점검", "분납컷오프점검"]] = "" # nan처리귀찮으니 ""로 초기화

# 채권자신고점검   # query는 정규식 표현 직접 사용 불가
채권자conds = ~미종결개시건.채권자목록.str.contains(검색용사명, regex=True)
미종결개시건.loc[채권자conds, "채권자신고점검"] = "채권자없음"

# 계좌신고점검
계좌신고검색어 = 검색용사명 + ".*계좌.*(신고|신청).*제출"
계좌신고conds = ~미종결개시건["진행내용"].str.contains(계좌신고검색어, regex=True)
미종결개시건.loc[계좌신고conds, "계좌신고점검"] = "계좌신고안됨"

# 분납컷오프일이 계좌컷오프일 이후인 경우 : 있을 수 없는일
분납컷오프오류cond = pd.to_datetime(미종결개시건.컷오프일,errors='coerce') < pd.to_datetime(미종결개시건.분납컷오프일,errors='coerce')
미종결개시건.loc[분납컷오프오류cond, "분납컷오프점검"] = "분납컷오프>계좌컷오프"


# 체크항목 합치기
미종결개시건["신고점검"] = 미종결개시건.채권자신고점검 + 미종결개시건.계좌신고점검 + 미종결개시건.분납컷오프점검
진행점검오류cols = 기본조회칼럼 + ["신고점검", "개시결정일", "변제계획인가일", "현재결과일"]
진행점검오류 = 미종결개시건.query('신고점검!=""')[진행점검오류cols]

# 출력
if len(진행점검오류) > 0 : 
    print(f'신고점검 오류 : {len(진행점검오류)}건. 엑셀출력')
    진행점검오류.to_excel(join(path_wd, "[check]개인회생_진행점검오류_"+cutoff+".xlsx"), index=False)
else : 
    print('진행 점검 이상무')
    
# 진행점검정상건이라 분납비고 삭제할 건 : 분납비고에 계좌신고, 채권자에 대한 내용이 있는 건
진행점검정상 = 미종결개시건[(미종결개시건.신고점검=="")&(미종결개시건.분납비고.str.contains(분납비고prefix, regex=True))][["분납키","채무자키","계좌키","분납비고"]]
진행점검정상.rename(columns={"분납비고":"구분납비고"},inplace=True)
진행점검정상["분납비고"] = "NULL"
진행점검정상 = 진행점검정상[["분납키","채무자키","계좌키","분납비고","구분납비고"]]

if len(진행점검정상) > 0 : 
    print(f'분납비고 삭제 체크 : {len(진행점검정상)}건. 엑셀출력')
    진행점검정상.to_excel(join(path_wd, "[전산수정]개인회생_분납비고삭제_진행점검_"+cutoff+".xlsx"), index=False)
else : 
    print('분납비고 삭제할 건 없음')

진행점검 오류 : 10건. 엑셀출력
분납비고 삭제 체크 : 1건. 엑셀출력


#### 미종결 인가건 정의

In [75]:
raw_data["상태및담당자"] = raw_data["채무상태"] + raw_data["담당자"]

# 변제오류무시 문구
변제오류무시 = "_" + cutoff[:4] + "변제오류무시"

# 미종결인가건 중 다음을 제외(분납메모) : 일반,제외,통합,이번달 변제오류무시건(변제오류 아님 : 변제일 미도래, 정정작업 예정이거나 이미 했더라도 해결까지 시간소요가 되는 것)
미종결인가건 = raw_data.query(
                        '분납상태 == "진행" and 인가미인가=="인가" and 현재결과!="종결" \
                        and ~(상태및담당자.str.contains(@종료) or 상태및담당자.str.contains(@종결요청)) \
                        and ~(분납메모.str.contains("제외|일반|통합") \
                        and ~(분납비고.str.contains(@변제오류무시))'
                        ) # not종결, not제외, not오류무시

# 정렬 : 관할법원,사건번호(주민번호는 차주번호임)
미종결인가건.sort_values(by=["관할법원","사건번호"], inplace=True)

# 현재회차정보, 현재회차차이 추가하기 ★★★★★
미종결인가건[["당사변제시작일", "현재회차변제일", "현재회차변제금","말일여부"]] = 미종결인가건.apply(lambda row : pd.Series(현재회차정보찾기(row)), axis=1)
미종결인가건["최근변제예정일"] = 미종결인가건.apply(lambda row: 최근변제예정일찾기(row, 조회일_dt), axis=1)
미종결인가건["현재회차차이"] = 미종결인가건.현재회차 - 미종결인가건.현재변제회차

#분납컷오프일체크용 전산-법원
# pandas가 자동으로 ZeroDivisionError를 infinte로 처리해주므로 이렇게 써도 됐던것
미종결인가건["출금대상회차"] = round(미종결인가건.출금대상잔액 / 미종결인가건.매월변제예정금액, 2)
미종결인가건["미납회차차이"] = round(미종결인가건.미납회차 - 미종결인가건.채무자미납회차 - 미종결인가건.출금대상회차, 2)

        # # '미납회차차이' 계산 함수 정의
        # def calculate_difference(row):
        #     try:
        #         return int(row['미납회차'] - row['채무자미납회차'] - (row['출금대상잔액'] / row['매월변제예정금액']))
        #     except ZeroDivisionError:
        #         return 0  # 또는 다른 적절한 값, 예: NaN, float('inf')

        # # '미납회차차이' 계산
        # 미종결인가건['미납회차차이'] = 미종결인가건.apply(calculate_difference, axis=1)


# # 참고용으로 볼거면 출력(변제조회 최신화 한 뒤에)
미종결인가건cols = list(dict.fromkeys(기본조회칼럼 + 차이칼럼 + 회차칼럼 + 법원회차칼럼 + 변제조회칼럼 + 은행칼럼 + 채권번호칼럼+회차정보칼럼 + 임시칼럼))
save_df_to_excel(미종결인가건[미종결인가건cols], join(path_wd, "[참고용]개인회생_미인가종결건_"+cutoff+".xlsx"))

#### 2.변제기본점검 : (인가) 필수입력값
- 분납비고 prefix = 변제기본_
- 변제현황서비스이용불가는 원래 기본점검에 포함하였으나 사실상 변제조회 된 것이므로 최신화성공여부만 살피고, 최신화 후 변제회차점검을 한다.

In [51]:
미종결인가건[["제외점검", "채권번호점검", "신고조회결과점검"]] = ""
분납비고prefix = "변제기본_"

# 1) 제외건류
제외conds = (미종결인가건.총분납금==0) & ((미종결인가건.최초원금==0)|(미종결인가건.현재원금==0)) & (~미종결인가건.분납메모.str.contains("제외|통합|일반"))
미종결인가건.loc[제외conds, "제외점검"] = "제외확인, "


# 2) 채권번호 오류
    # 첫 번째 조건: 변제채권번호가 빈 문자열이고 변제채권번호스캔이 빈 문자열이 아닌 경우
채권번호cond1 = (미종결인가건['변제채권번호'] == "") & (미종결인가건['변제채권번호스캔'] != "")
    # 두 번째 조건: 변제채권번호스캔이 변제채권번호를 포함하지 않는 경우
채권번호cond2 = ~미종결인가건.apply(lambda row: row['변제채권번호'] in row['변제채권번호스캔'], axis=1)
    # 두 조건을 합쳐서 불리언인덱싱
채권번호conds = 채권번호cond1|채권번호cond2
미종결인가건.loc[채권번호conds, "채권번호점검"] = "채권번호확인, "

# 3) 신고조회결과(변제조회 이용가능) : 계좌정보오등록은 3)과 중복될 수 있음
조회결과conds = [
    미종결인가건.신고조회결과.str.match('은행코드'), # 변제조회 오류
    미종결인가건.신고조회결과.str.match('채권자'), # 변제조회는 되나 당사 정보 없음
    # 필수항목오류 : 위 두 조건에서 이미 걸러질 것이나 혹시 몰라 한 번 더
    미종결인가건.관할법원 == "",
    미종결인가건.사건번호 == "",
    (미종결인가건.신고은행 == "")|(미종결인가건.신고은행!=미종결인가건.입금은행),
    (미종결인가건.신고계좌 == "")|(미종결인가건.신고계좌!=미종결인가건.입금계좌),
    (~미종결인가건.신고채권자.str.match("헬프|솔림"))|(미종결인가건.신고채권자!=미종결인가건.입금채권자)
    ]
조회결과values = [
    "조회결과_계좌정보오등록, ",
    "조회결과_채권자현황오류, ",
    "필수항목_관할법원없음, ",
    "필수항목_사건번호없음, ",
    "필수항목_신고은행오류, ",
    "필수항목_신고계좌오류, ",
    "필수항목_신고채권자오류, "
    ]
미종결인가건['신고조회결과점검'] = np.select(조회결과conds, 조회결과values, default="")

# 후처리
미종결인가건["변제기본점검"] = 미종결인가건.제외점검 + 미종결인가건.채권번호점검 + 미종결인가건.신고조회결과점검

변제기본점검cols = list(dict.fromkeys(기본조회칼럼 +["변제기본점검", "변제계획인가일"] + 변제조회칼럼 + 은행칼럼 + 채권번호칼럼 + 회차칼럼 + 차이칼럼 + 회차정보칼럼))
변제기본점검오류 = 미종결인가건.query('변제기본점검!=""')[변제기본점검cols]

변제기본점검오류.sort_values(by=['변제기본점검'], inplace=True)

# 출력
if len(변제기본점검오류) > 0 : 
    print(f'제외,채권번호,필수항목 오류 : {len(변제기본점검오류)}건. 엑셀출력')
    변제기본점검오류.to_excel(join(path_wd, "[check]개인회생 기본점검사항 오류_"+cutoff+".xlsx"), index=False)
else : 
    print('기본 점검 이상무')
    
# 정상건이라 분납비고 삭제할 건     # 오류아님      # 분납비고에 변제기본 오류에 대한 내용 있음
변제기본점검정상 = 미종결인가건.query('변제기본점검=="" and 분납비고.str.contains(@변제기본prefix)')[["분납키","채무자키","계좌키","분납비고"]]
변제기본점검정상.rename(columns={"분납비고":"구분납비고"},inplace=True)
변제기본점검정상["분납비고"] = "NULL"
변제기본점검정상 = 변제기본점검정상[["분납키","채무자키","계좌키","분납비고","구분납비고"]]

if len(변제기본점검정상) > 0 : 
    print(f'분납비고 삭제할 건 : {len(변제기본점검정상)}건. 엑셀출력')
    변제기본점검정상.to_excel(join(path_wd, "[전산수정]개인회생_분납비고삭제_인가기본점검"+cutoff+".xlsx"), index=False)
else : 
    print('분납비고 삭제할 건 없음')

제외,채권번호,필수항목 오류 : 19건. 엑셀출력
분납비고 삭제할 건 : 17건. 엑셀출력


#### 3.변제최신화실패 : 변제최신화일 < 컷오프일
- 사건조회 메뉴를 통해 최신화 돌리기
- 여기까지 하고 나서 전산 다시 다운로드 후 진짜 자산정리
- 이후 체크사항은 자산정리 당일에 할 내용은 아니다.

In [87]:
# 아침에 정상적으로 최신화 돌았으면 전날(컷오프) 법원 업무종료시까지의 데이터가 업데이트된다.
# 따라서 신고조회일이 컷오프일보다 작은 경우만 최신화 후보
# 그중 현재회차변제예정일이 컷오프일 근처인 것은 최신화가 반드시 필요(250개 하는데 15분 소요 > 필요한 것만 하자)
# 컷오프는 조회 전날이므로 신고조회일이 컷오프일보다 작으면서 변제예정일이 3일 이내인 것만 하자.

# 타입변경
미종결인가건['신고조회일'] = pd.to_datetime(미종결인가건['신고조회일'])
미종결인가건['최근변제예정일'] = pd.to_datetime(미종결인가건['최근변제예정일'])

# 변제예정일과 조회일의 차이가 3일 이내
미종결인가건['날짜차이'] = (미종결인가건['최근변제예정일'] - 미종결인가건['신고조회일']).abs().dt.days

# 변제조회 최신화 필요건
최신화cond1 = 미종결인가건["신고조회일"] < cutoff_dt # 컷오프 실패건 or 주말껴서 컷오프 안 된 건(주말이라도 법원회차계산 되어야지)
# 변제현황조회서비스불가건이라도 미납회차와 출금예정금액 변화가 있을 수 있으므로 최신화 필요함
# 최신화cond2 = ~미종결인가건["변제현황조회"].str.match("변제현황조회") # 변제현황조회서비스이용불가건은 입금없는 건이므로 법원회차 최신화의 의미가 없다.(변제중이라도 모두 미납)
최신화cond_special = 미종결인가건['날짜차이'] <= 3 # 컷오프 == 조회일 일때

# 컷오프일과 조회일이 같으면 날짜차이 상관없이 모두 최신화 대상(현재회차 등 업데이트 해야 하므로)
# 그러나 중간에 하면 당연히 오차가 발생하게 되므로 법원업무 끝나고 하거나, 전날을 기준으로 해야 한다.
# 업무 중간에 하는 것은 그냥 테스트용으로나
if cutoff_dt == 조회일_dt : 
    최신화conds = 최신화cond1 & 최신화cond_special
# 이 경우가 일반적인 경우
else : 
    최신화conds = 최신화cond1

# 변제조회최신화대상 = 미종결인가건.query("신고조회일 != @조회일")
변제조회최신화대상 = 미종결인가건.loc[최신화conds]

# 사건조회 파일 만들기 
사건조회cols = ["사건키", "관할법원", "사건번호", "당사자", "채권자", "신고은행", "신고계좌", "신고채권자", "변제채권번호"]
사건조회 = pd.DataFrame(None, columns=사건조회cols)
사건조회["관할법원"] = 변제조회최신화대상.관할법원
사건조회["사건번호"] = 변제조회최신화대상.사건번호
사건조회["당사자"] = 변제조회최신화대상.분납자성명인
사건조회["채권자"] = "헬프"
사건조회["신고은행"] = 변제조회최신화대상.신고은행
사건조회["신고계좌"] = 변제조회최신화대상.신고계좌
사건조회["신고채권자"] = "헬프"
사건조회["변재채권번호"] = 변제조회최신화대상.변제채권번호입력

# 엑셀저장
사건조회.to_excel(join(path_wd, "개인회생사건조회_"+조회일+".xlsx"), index=False)

### 일괄 최신화 후에는 변제오류가 있더라도 신고조회일을 바뀐다.
### 따라서 신고조회일로 오류 따지는 것은 더이상 의미없고, 신고결과 등을 봐야한다.

,채무자키,분납키,채권구분,매각사구분,계좌키,Unnamed: 5,관할법원,인가미인가,현재결과,분납상태,...,파산선고결정일,상태및담당자,실변제일,실변제금,말일여부,최근변제예정일,현재회차차이,미납회차차이,회차수정오류여부,날짜차이
6979,20527807,20136068,IRL(인가),카카오-01,201061472,0,강릉지원,인가,인가,진행,...,,개인회생(확정)개인회생,05,52952,,2024-07-05,0,-1.01,,2.0
6980,20525801,20136223,IRL(인가),카카오-01,201059466,0,강릉지원,인가,인가,진행,...,,개인회생(확정)개인회생,05,146857,,2024-07-05,0,-1.01,,2.0
6766,20525866,20137038,IRL(인가),카카오-01,201059531,0,강릉지원,인가,인가,진행,...,,개인회생(확정)개인회생,05,277271,,2024-07-05,0,-1.00,,2.0
5997,20514670,20130910,IRL(인가),퀀텀-01,201045523,0,강릉지원,인가,인가,진행,...,,개인회생(확정)개인회생,01,181272,,2024-07-01,0,0.00,,2.0
7796,20515418,20131130,IRL(인가),퀀텀-01,201046271,0,광주지방법원,인가,인가,진행,...,,개인회생(확정)개인회생,01,200736,,2024-07-01,0,-1.00,,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860,20452487,20120044,CCRS,고려저축-01,200975703,0,춘천지방법원,인가,인가,진행,...,,개인회생(확정)개인회생,01,10231,,2024-07-01,0,-0.00,,2.0
2849,20440936,20119864,NPL,KB카드-01,200962295,0,춘천지방법원,인가,인가,진행,...,,개인회생(확정)개인회생,06,323969,,2024-07-06,0,-0.01,,3.0
2850,20440935,20119863,NPL,KB카드-01,200962294,0,춘천지방법원,인가,인가,진행,...,,개인회생(확정)개인회생,06,255335,,2024-07-06,0,-0.01,,3.0
2707,20452004,20108920,IRL(미인가),고려저축-01,200974769,0,춘천지방법원,인가,인가,진행,...,,개인회생(확정)개인회생,03,3764,,2024-07-03,0,-0.00,,0.0


In [ ]:
# 칼럼정의
차이칼럼 = ["현재회차차이", "미납회차차이"]
회차칼럼 = ["현재회차", "현재변제회차", "잔여회차", "납입회차", "미납회차", "총분납회차", "초과입금"]
변제조회칼럼 = ["변제현황조회","신고조회일","신고조회결과","신고은행","신고계좌"]
채권번호칼럼 = ["변제채권번호입력", "변제채권번호스캔", "변제채권번호"]
회차정보칼럼 = ["실변제일", "실변제금"]
회차구간칼럼 = ["변제회차1", "변제기간1", "변제회차2", "변제기간2", "변제회차3", "변제기간3", "회차수정오류여부"]

#### 4.변제회차점검-변제결과(회차,금액,분납컷오프일) 점검
- 전산 마지막 다운로드 후 작업, 그래도 오류 나오면 그건 자산정리 전에는 오류 수정불가이므로 그냥 진행(?)
- 현재회차가 동일한 경우, 미납회차차이 = -(납입회차차이)

In [37]:
# 회차오류 : 총회차, 현재회차, 미납회차, *납입회차(현재-(미납+출금예정))

# 범위한정 : 변제조회정상, 당사변제일도래
변제정상건 = 미종결인가건.query('변제기본점검=="" and 당사변제시작일>"20"+@cutoff') # 변제서비스이용불가라도 변제조회 됐으면 포함됨

# 전처리
변제정상건['분납컷오프일'] = pd.to_datetime(변제정상건['분납컷오프일'], errors='coerce') # errors='coerce' 변환할 수 없는 값은 NaT(Not a Time)으로
변제정상건['컷오프일'] = pd.to_datetime(변제정상건['컷오프일'], errors='coerce')
변제정상건['변제시작일'] = pd.to_datetime(변제정상건['변제시작일'], errors='coerce')
변제정상건['변제계획인가일'] = pd.to_datetime(변제정상건['변제계획인가일'], errors='coerce')

변제정상건["총분납회차차이"] = 변제정상건.총분납회차 - 변제정상건.전체변제회차
    # 컷오프확인대상 : 인가일과 변제시작일 이후 매입(컷오프)한 건. 스케줄 중간부터 시작. 분납시작일은 수기등록할 수 있으므로 변제시작일 사용
    # 당사변제시작일은 이미 컷오프일에 영향을 받은 것이므로 사용하면 안 된다.
cond = (변제정상건['분납컷오프일'].notna()) | (변제정상건['컷오프일'] > 변제정상건[['변제시작일', '변제계획인가일']].max(axis=1))
변제정상건['컷오프확인대상'] = np.where(cond, "확인대상", "")

# 점검
변제오류conds = [
    (변제정상건.총분납회차!=변제정상건.전체변제회차) & (~변제정상건.분납비고.str.contains('변제회차_총회차오류아님')), # 오류아닌거 확인 안 한 건 중에 총회차 차이 나는 건
    (변제정상건.납입회차==0) & (변제정상건.현재변제회차-변제정상건.채무자미납회차>=1), # 채무자 입금 했는데, 초회입금 안 된 건
    (변제정상건.현재회차차이 > 1), 
    (변제정상건.현재회차차이 == 1)&(변제정상건.최근변제예정일 != 조회일), # 조회일은 str yyyy-mm-dd
    # 미납회차 관련 시작--------------
    # 당사 1회차부터 : 분납컷오프 상관 없는 거. 회차차이가 [1]이하인 경우는 pass
    (변제정상건.컷오프확인대상=="") & (변제정상건.미납회차차이>1) & (변제정상건.출금대상회차 >= 1), # 입금요청건
    (변제정상건.컷오프확인대상=="") & (변제정상건.미납회차차이>1) & (변제정상건.출금대상회차 < 0), # <1을 사용하지 않은것은 채무자미납회차만으로 1회차 이상 차이나는 경우를 확인하기 위해, 발생해선 안 되는 경우
    (변제정상건.컷오프확인대상=="") & (변제정상건.미납회차차이<1), # 초과입금발생 : 실질적인 의미는 선입금, 오입금, 계좌매칭오류 일 수 있음
    # 분납컷오프일 조정대상
    (변제정상건.컷오프확인대상!="") & (변제정상건.미납회차차이<-1), # 전산미납회차가 적으니 늘리기 위해 분납컷오프일 앞으로 당겨야. 드물게 초과입금
    (변제정상건.컷오프확인대상!="") & (변제정상건.미납회차차이>1) # 분납컷오프 뒤로
    ]
변제오류values = ["총회차다름", "입금요청_초회부터미입금", "현재회차차이발생2~","현재회차차이발생1", 
              # 미납회차 관련
              "전회차_입금요청",
              "전회차_출금대상금액도없는데 미납회차차이",
              "전회차_초과(선)입금", # 소액 있을 수는 있으나, 상당금액이 초과입금되면 변제계획이 수정되었어야 하므로 변제오류에서 검색이 됐어야 한다.
              "분납컷오프_앞으로or초과입금",
              "분납컷오프_뒤로"
              ]

변제정상건["변제오류체크"] = np.select(변제오류conds, 변제오류values, default="")

변제오류cols = [기본조회칼럼 + 차이칼럼 + 회차칼럼 + 법원회차칼럼 + 회차정보칼럼]
변제오류 = 변제정상건.query('변제오류체크 != ""')[변제오류cols]

if len(변제오류) > 0 :
    print(f"변제회차오류 {len(변제오류)}건 발생. 엑셀로 저장합니다.")
    변제오류.to_excel(join(path_wd, "[check]개인회생 변제회차오류_"+cutoff+".xlsx"))
else : 
    print(f'변제회차오류 의심건이 없습니다.')
    
###### 해당 목록 모두 전산에서 조회 후 변제최신화 눌러준뒤 처음부터 재작업 ######
# 회차차이 1이고 현재회차에 맞는 변제 예정일과 조회일의 평일 차이가 하루차이 이하이면 문제 없는 경우다.(컷오프 조정도 필요 없음)
# 전산은 예정일이 되면 모든 회차가 바로 업데이트되고, 법원은 예정일이 지난 평일에나 업데이트가 되기 때문에
# 그외의 것들은 분납메모 및 메모 보면서 개별 확인 해야

변제오류 8건 발생. 엑셀로 저장합니다.


#### 동일사건,다른 회차 - 현재회차2

In [10]:
# '관할법원+사건번호'로 그룹화 : 주민번호는 분납자가 아니라 채무자라서 정확하지 않음
# '현재회차'고유값이 둘 이상인 건
# '주민번호인'으로 그룹화하고 '현재회차'의 고유값이 2개 이상인 그룹 필터링
동일사건다른회차 = 미종결인가건.groupby(by=["관할법원", "사건번호"]).filter(lambda x: x['현재회차'].nunique() >= 2)
동일사건다른회차cols = [기본조회칼럼 + 차이칼럼 + 회차칼럼 + 법원회차칼럼 + 회차정보칼럼]

if len(동일사건다른회차) : 
    print(f"동일사건에 현재회차가 다른 경우 {len(동일사건다른회차)}건. 엑셀 확인")
    동일사건다른회차[동일사건다른회차cols].to_excel(join(path_wd, "[check]개인회생_동일사건다른회차_"+cutoff+".xlsx"), index=False)
else :
    print("이상무")


이상무


#### 분납상태 중단으로 변경할 건
- 분납상태 진행으로 바꿀것 : 중단 & ~매각,환매 & 최근1개월 개인회생 입금조회

In [40]:

분납상태중단할건 = raw_data.query('분납상태 == "진행" and (상태및담당자.str.contains(@종료) or 상태및담당자.str.contains(@종결요청))')
if len(분납상태중단할건)> 0 : 
    print(f"분납상태 중단으로 변경할 것이 있습니다. {len(분납상태중단할건)}건")
    분납상태중단할건.to_excel(join(path_wd, "[check]개인회생 분납상태 중단할것_"+cutoff+".xlsx"), index=False)
else : 
    print("분납상태 중단할 게 없습니다.")

분납상태 중단으로 변경할 것이 있습니다. 56건


#### 수동입금 대상 : 사건번호(주민번호x)동일, 변제액 동일, 배수관계
- 입금파일에는 사건번호만 있으므로 사건번호로만 그룹화
- 다회차가 들어와서 배수관계가 되는 건은 특정할 수 없으므로 사후확인

In [42]:
def has_multiple_relationship(series):
    """현재회차변제금들이 서로 배수 관계인지 확인"""
    non_zero_values = [x for x in series if x > 0]
    if len(non_zero_values) < 2:
        return False
    sorted_series = sorted(non_zero_values)
    
    # 각 값을 base로 시도
    for i in range(len(sorted_series)-1):
        base = sorted_series[i]
        for other in sorted_series[i+1:len(sorted_series)]:
            if other % base == 0 : 
                return True
    return False

grouped = 변제정상건.groupby(by=["사건번호"])

# 배수 관계에 있는 사건번호 찾기
multiples = grouped['현재회차변제금'].apply(has_multiple_relationship)
multiples_cases = multiples[multiples].index.tolist()


# 엑셀 조회
배수cols = [기본조회칼럼 + 회차칼럼 + 회차정보칼럼]
변제금배수 = 변제정상건[변제정상건['사건번호'].isin(multiples_cases)][배수cols]
변제금배수.to_excel(join(path_wd, "[check]개인회생_변제금배수(입금유의)_"+cutoff+".xlsx"))

#### 금액 체크(초과입금)
- 단순히 초과입금 발생건을 모두 선택한 것으로 위의 오류점검결과를 원점에서 점검하는 용도로 쓰다가 안정성 확인되면 할 필요 없음

In [77]:
# 금액 체크 - 초과입금은 폐지 진행중인 경우에도 발생할 수 있으므로 raw_data를 사용한다.
# 입금이 잘못된 경우 초과입금이 생기는데, 이를 더해주지 않고 있으므로 - 값이 나올 수 있다.
# 초과입금은 회차별 변제금액 이외의 금액이 있는 경우 발생한다.(ex-3만원 변제인데 4만원 입금들어온경우)

# 조회칼럼
추가칼럼 = ["총분납금","총분납입금","분납미납금","분납잔금","총분납잔금","초과입금","총분납회차","현재회차","납입회차","잔여회차","미납회차",
        "매월변제예정금액","출금대상잔액","출금대상기준일","채무자미납회차","재산처분예정금액","전체변제회차","현재변제회차","잔여변제회차","재산처분납입금액",
        "컷오프일","분납컷오프일"]
초과입금칼럼 = list(dict.fromkeys(기본조회칼럼 + 추가칼럼))

# 초과입금은 회차별 변제금액 이외의 금액이 있는 경우 발생한다.(과거 미납회차, 은닉재산 발견, 선입금)
초과입금 = raw_data.query('분납상태=="진행" and 초과입금>0')[초과입금칼럼]
# 총분납금 < 총분납입금 : 말 그대로의 초과입금 (위에 포함됨)
총분납금초과 = raw_data.query('분납상태=="진행" and 총분납금<총분납입금')[초과입금칼럼]

display(초과입금.head(3))
display(총분납금초과.head(3))

# 엑셀출력
초과입금.to_excel(join(path_wd,"[check]개인회생 초과입금_"+cutoff+".xlsx"), index=False)
총분납금초과.to_excel(join(path_wd,"[check]개인회생 입금이 총분납금을 초과_"+cutoff+".xlsx"), index=False)

,채무자키,계좌키,분납키,분납자관계,분납자성명인,주민번호인,관할법원,사건번호,인가미인가,현재결과,...,출금대상잔액,출금대상기준일,채무자미납회차,재산처분예정금액,전체변제회차,현재변제회차,잔여변제회차,재산처분납입금액,컷오프일,분납컷오프일
138,10924911,1868391,1340371,채무자,서은주,731121-2495717,인천지방법원,2022개회8225,인가,인가,...,103249,2024-06-29,0.0,0,36,23,13,0,2022-09-02,
160,10924981,1868461,1340441,채무자,이상진,640103-1786015,대구지방법원,2022개회10028,인가,인가,...,22858,2024-06-29,0.0,0,36,24,12,0,2022-09-02,
228,10924936,1868416,1340396,채무자,,820528-1696813,대구지방법원,2022개회209200,인가,인가,...,1388,2024-06-29,0.0,0,36,25,11,0,2022-09-02,


,채무자키,계좌키,분납키,분납자관계,분납자성명인,주민번호인,관할법원,사건번호,인가미인가,현재결과,...,출금대상잔액,출금대상기준일,채무자미납회차,재산처분예정금액,전체변제회차,현재변제회차,잔여변제회차,재산처분납입금액,컷오프일,분납컷오프일
138,10924911,1868391,1340371,채무자,서은주,731121-2495717,인천지방법원,2022개회8225,인가,인가,...,103249,2024-06-29,0.0,0,36,23,13,0,2022-09-02,
478,10830762,1767268,1279551,채무자,차광민,840215-1069721,서울회생법원,2021개회1060051,인가,인가,...,362548,2024-07-01,0.0,0,36,34,2,0,2022-03-11,2022-03-11
522,10924669,1868148,1340128,채무자,강승범,661120-1068212,인천지방법원,2021개회1015847,인가,인가,...,11901,2024-06-29,1.0,0,36,35,1,0,2022-09-02,


#### 일반

In [115]:
# 진행건 중 체크 건 있는지 확인
수작업idx = raw_data.query("인가미인가.str.contains('체크') and 분납상태 =='진행'").index
if len(수작업idx) > 0: 
    display(raw_data.loc[수작업idx,["채무자키","계좌키","분납키","주민번호인","인가미인가","현재결과","분납메모","항고일","변제계획인가일"]])
    #수작업idx.to_excel("xxx.xlsx")
else : 
    print("진행건중 체크할 거 없음")

진행건중 체크할 거 없음


In [ ]:
# # # 확인 후 수정 or pass                               ####################
# raw_data.loc[수작업idx,["인가미인가"]] = ["미인가"]
# raw_data.loc[수작업idx,["분납메모","인가미인가"]] = ["미결정", "미인가"]
#                                                       ####################

In [116]:
# check ----------------------------
cols = ["채무자키", "분납키", "분납자관계", "분납자성명인", "인가미인가", "현재결과","현재결과일", "분납상태", "분납메모","중단사유", "분납중단일", "채무상태", "담당자", "개시결정일", "변제계획인가일", "사건번호"]
check = raw_data[cols].copy()
# 인가미인가 : 인가,미인가 // 폐지,면책,이송 // 체크
# 분납상태 : 진행, 중단

# 1. 인가미인가와 분납상태의 불일치 -> 인가 or 미인가 = 분납상태 진행
# 1-1.진행건 vs 상태 중단 : 중단사유 있는지?
중단사유적어 = check.query('인가미인가.str.contains("인가|미인가") & 분납상태=="중단" & 중단사유 == ""')
if len(중단사유적어) > 0 :
    print("중단사유 명시 필요") # 현재결과는 중단사유가 아니라 바로 확인 불가
    display(중단사유적어)
    
# 1-2. 중단건(인가/미인가 아닌것) vs 상태 진행
진행건_인가미인가오류 = check.query('분납상태=="진행" & ~인가미인가.str.contains("인가|미인가")')
if len(진행건_인가미인가오류) :
    print("진행건인데 인가나 미인가가 아닌 것 있음. 폐지 맞는지 확인(폐지결정 나더라도 바로 중단으로 하지 않는다. 2024-03~ / 인가미인가가 맞으면 됨.)")
    display(진행건_인가미인가오류.sort_values(by=['인가미인가']))

# 2. not중지건 중 인가전 현재결과 vs 변제계획인가일 있는 경우
미인가건임 = check.query('분납상태=="진행" and 인가미인가=="미인가" and 변제계획인가일!=""')    ############## 20149032 는 회단사건으로 인가일 표기위치가 다름
인가건임 = check.query('분납상태=="진행" and 인가미인가=="인가" and 변제계획인가일=="" and 분납키!="20149032" and ~(분납메모.str.contains("일반회생"))')
if len(미인가건임) > 0 :
    print("미인가건인데 왜 인가일이 있음?")
    display(미인가건임)

if len(인가건임) > 0 :
    print("인가건인데 왜 인가일이 없음?, 종결건, 확인하지 않은 일반회생은 그럴 수 있음(분납메모 작성필요)")
    display(인가건임)

# 3. not중지건 중 개시결정일 있는데 전단계인 경우
# (반대)왜인지는 몰라도 개시결정일을 못 읽는 경우가 1~2건 있네.. 
개시건임 = check.query('분납상태=="진행" and 개시결정일 != "" and 현재결과.str.contains("금지|접수")')
개시건인데 = check.query('분납상태=="진행" and 개시결정일 == "" and 현재결과.str.contains("개시") and 분납키!="20149032"')
if len(개시건임) > 0 :
    print("개시일 있는데 왜 전단계? - 항고건은 분납메모를 통해서 ")
    display(개시건임)
    
if len(개시건인데) > 0 :
    print("개시인데 왜 개시결정일 없음? 분납키 20145963, 20149575 개시임은 확인했는데 결정일을 못 읽네..")
    display(개시건인데)

# 4. not중지건 일반회생(회단,간회단) 중 분납메모 없는 건
미확인일반회생 = check.query('사건번호.str.contains("회단") and 분납상태=="진행" and 현재결과=="인가" and ~(분납메모.str.contains("일반회생"))')
if len(미확인일반회생) > 0 :
    print("인가된 일반회생인데 메모 왜 없니")
    display(미확인일반회생)

# 항고, 폐지취소 건 중 분납중지건은 확인 완료 (240130)
# 최신화 불가건 : 항고 뿐임 (폐지취소는 최신화 됨, 폐지 후 과거의 인가/미인가 여부를 확인하는 작업이 필요할뿐)
# 담당자, 채무상태, 누락 건 등은 자산정리 후 해야 정확

진행건인데 인가나 미인가가 아닌 것 있음. 폐지 맞는지 확인(폐지결정 나더라도 바로 중단으로 하지 않는다. 2024-03~ / 인가미인가가 맞으면 됨.)


,채무자키,분납키,분납자관계,분납자성명인,인가미인가,현재결과,현재결과일,분납상태,분납메모,중단사유,분납중단일,채무상태,담당자,개시결정일,변제계획인가일,사건번호
3798,20410183,20106866,채무자,이정민,면책,면책허가결정,2024-06-07,진행,,,,개인회생(면책),개인회생,2021-08-27,2022-01-10,2021개회207958
5491,20514735,20131149,채무자,이미경,면책,면책허가결정,2024-03-05,진행,,,,개인회생(면책),개인회생,2020-12-21,2021-02-23,2020개회232114
94,20464550,20150057,채무자,윤성숙,폐지,절차폐지결정일,2024-06-20,진행,,,,개인회생(폐지진행중),도현관,2024-05-20,,2024개회106840
235,20525642,20151108,채무자,이장환,폐지,절차폐지결정일,2024-06-26,진행,,,,개인회생(폐지진행중),배정(대기),2024-04-02,,2023개회249564
257,20454938,20145958,채무자,정도경,폐지,폐지결정,2024-06-24,진행,,,,개인회생(폐지진행중),박준호,2024-04-29,,2023개회1122168
841,20472327,20133345,채무자,최신규,폐지,절차폐지결정일,2024-06-24,진행,,,,개인회생(폐지진행중),박준호,2022-12-05,2023-02-28,2022개회1111369
2186,20501067,20130525,채무자,김형진,폐지,절차폐지결정일,2024-06-24,진행,,,,개인회생(폐지진행중),종결(매각),2022-02-03,2022-04-04,2021개회1110932
2415,20452763,20141598,채무자,이경선,폐지,폐지결정,2024-06-25,진행,,,,개인회생(폐지진행중),배정(대기),2023-09-21,,2021개회230637
4644,20514529,20131147,채무자,이명주,폐지,절차폐지결정일,2024-06-17,진행,,,,개인회생(면책),박준호,2021-05-11,2021-12-07,2020개회20225
4952,20452377,20120032,채무자,김영화,폐지,절차폐지결정일,2024-06-27,진행,,,,개인회생(면책),배정(대기),2022-10-12,2023-03-16,2020개회235819


In [ ]:
# ### 체크 후 수정
# raw_data.loc[[14],["개시결정일","인가미인가"]] = ["파기환송", "미인가"]
# raw_data.loc[14,"개시결정일"] = '2024-04-02'

In [117]:
# 일반내용 ~ 필터제출기준현재회차 내용삭제
ind1 = raw_data.columns.get_loc('일반내용')
ind2 = raw_data.columns.get_loc('필터제출기준현재회차')
raw_data.iloc[:,ind1:ind2] = ""

### 특이건 회차,금액 정리

#### 통합신고,제외

In [118]:
# 통합신고, 제외
# 통합신고는 애초에 하나의 계좌로 금액을 합산하여 신고한 경우로 신고채권번호도 한 개. 
#       단일채무자키인 경우 전산 통합등록하고, 아무계좌에서나 변제처리 되어도 상관없다. - 통합등록한 경우 계좌 분리과정에서 대표계좌만 금액이 남는다.
#       다채무자키인 경우 통합등록은 불가, 따라서 대표가 아닌 계좌의 분납메모에는 반드시 통합신고 적어줘야 금액 0원처리 가능
# 제외는 계좌별로 신고하였으나, 원금이 적어 총분납금이 0원인 채권으로 신고채권번호는 여러개이나, 법원채권번호는 하나만 나온 경우
#       신규건은 분리등록이 원칙. 이미 통합등록이 된 경우 분리등록해줘야 하나 그대로 두고 제외가 아닌 통합신고로 처리해도 별 문제는 없음
####### 통합신고채권(채권번호 하나)은 계좌별로 등록할 필요가 없으니 이렇게 적을 필요가 없음(아래서, 계좌통합건은 행분열하고 모두 0원으로 처리, 현재원금은 유지)
####### 통합신고채권은 계좌별로 분리하지 말고 통합해서 전산등록해야

#240131 정정, 회차 및 회차별 변제예정액은 그대로 뒀다. 분납메모만 봐도 되니까.
raw_data.loc[raw_data.분납메모.str.contains("통합|공채권|제외"), ["총분납금","총분납입금", "분납미납금", "분납잔금", "총분납잔금"]] = 0 # 입금내역 있는 것 재입금처리 해야함..

# 이전 코드
# raw_data.loc[raw_data.분납메모.str.contains("통합|공채권|제외"), ["총분납금","분납미납금"]] = 0
# raw_data.loc[raw_data.분납메모.str.contains("통합|공채권|제외"), ["분납잔금","총분납잔금"]] = raw_data['총분납금'] - raw_data["총분납입금"]

# # 엑셀 파일로 저장
# if not os.path.exists(path_result):
#     with pd.ExcelWriter(path_result, mode='w', engine='openpyxl') as writer:
#         raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)
# else : 
#     with pd.ExcelWriter(path_result, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer :
#         raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)

In [125]:
# 공채권류 확인
raw_data.loc[raw_data.분납메모.str.contains("(공채권|통합|제외)"),["채무자키","분납키","주민번호인", "분납메모", "총분납금","총분납입금", "분납미납금", "분납잔금", "총분납잔금"]].info() #.to_excel(join(path_wd, "공채권.xlsx"))

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 196 to 7811
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   채무자키    59 non-null     object
 1   분납키     59 non-null     object
 2   주민번호인   59 non-null     object
 3   분납메모    59 non-null     object
 4   총분납금    59 non-null     int64 
 5   총분납입금   59 non-null     int64 
 6   분납미납금   59 non-null     int64 
 7   분납잔금    59 non-null     int64 
 8   총분납잔금   59 non-null     int64 
dtypes: int64(5), object(4)
memory usage: 4.6+ KB


In [ ]:
######## 예전 수작업 내용 (지금은 모두 자동화했으므로 pass)
# 전산조회 할 거 작업 후
# - 분납상태-[진행] & 
# - 현재결과-[폐지취소결정, 폐지취소결정송달, 빈셀] & 
# - 인가미인가-[기타, #N/A, ""] 즉 인가,미인가가 아닌 거
# - 매각 제외 전이므로, 채무상태 및 담당자에서 종결, 매각건 제외

#### 연납건
- 분납메모가 있어야만 작성할 수 있다.

In [120]:
# 연납건 중 인가건 처리-----------------------------------------------------------------------------------------------------------------------------------
# 나우리의 계산법과 다르게 컷오프전 입금회차를 포함함.(즉 언제나 1회차 부터계산)
연납건index = raw_data.query('분납메모.str.contains("일반회생") and 분납상태=="진행"').index

if re.search("\d{6,6}",cutoff) :
    real_cutoff = cutoff
else : real_cutoff = datetime.strftime(datetime.today(), "%y%m%d")

for i in 연납건index : 

    기본정보 = raw_data.loc[i,"분납메모"].split("_")
    시작일 = datetime.strptime(raw_data.loc[i,"분납시작일"], "%Y-%m-%d")
    종료일 = datetime.strptime(raw_data.loc[i,"분납종료일"], "%Y-%m-%d")

    # 회차
    총분납회차 = 종료일.year - 시작일.year + 1
    if real_cutoff[2:] < str(시작일.month)+str(시작일.day) : 
        현재회차 = int("20"+real_cutoff[:2]) - 시작일.year # ex) 2022년 9월 30일, 분납시작일 2021년 12월 1일 > 1회차
    else : 
        현재회차 = int("20"+real_cutoff[:2]) - 시작일.year + 1 # ex) 2022년 12월 30일, 분납시작일 2021년 12월 1일 > 2회차
    납입회차 = int(기본정보[1]) + raw_data.loc[i,"매입전변제금"] # 솔림입금회차 + 매입전 입금회차
    잔여회차 = 총분납회차 - 현재회차
    미납회차 = 현재회차 - 납입회차

    # 금액
    _회차별_변제금_ = 기본정보[-1].split(",")

    회차별_변제금 = pd.Series(index=range(1,총분납회차+1), name="회차별변제금")
    for v in _회차별_변제금_ :
        회차구간, 회차구간별변제금 = v.split(":")
        회차구간별변제금 = int(회차구간별변제금)

        if re.search("~", 회차구간) : 
            sn = int(회차구간.split("~")[0])
            en = int(회차구간.split("~")[1])
            회차별_변제금.loc[sn:en] = 회차구간별변제금
        else : 
            회차별_변제금.loc[int(회차구간)] = 회차구간별변제금

    총분납금 = 회차별_변제금.sum()
    총분납입금 = 회차별_변제금.loc[:납입회차].sum()
    총분납잔금 = 총분납금 - 총분납입금 # = 분납잔금 + 분납미납금 
    분납잔금 = 회차별_변제금.loc[현재회차+1 : ].sum() # 미래
    분납미납금 = 총분납금 - 총분납입금 - 분납잔금 # 과거 # 구간별 변제금이 다를 수 있으므로 미납회차*변제금을 할 수 없음

    raw_data.loc[i, "총분납회차"], raw_data.loc[i, "현재회차"], raw_data.loc[i, "납입회차"], raw_data.loc[i, "잔여회차"], raw_data.loc[i, "미납회차"] = 총분납회차, 현재회차, 납입회차, 잔여회차, 미납회차
    raw_data.loc[i, "총분납금"],raw_data.loc[i, "총분납입금"], raw_data.loc[i, "분납미납금"], raw_data.loc[i, "분납잔금"], raw_data.loc[i, "총분납잔금"] = 총분납금, 총분납입금, 분납미납금, 분납잔금, 총분납잔금


### 반영OPB 및 요약

In [121]:
# OPB : 현재법비용 추가일 2024-04-03
# 240613 : a)초과입금이 대부분 매입전 미납회차입금라면 전산 그대로 가는게 맞고, b)채무자의 숨겨진 재산이 발견된 경우라도 과거의 변제스케줄이 달라지는 거니까 전산 그대로 가는게 맞고, 
# c)채무자가 미리 상환을 해서 미래 회차가 선입금 된 경우라면 그때만 빼주는 게 맞는데, 이 경우는 현재회차도 같이 바뀌는지도 확인을 해봐야 함
# 일단은 a)가 중심이라 가정하고 전산 그대로 가자.
raw_data['반영OPB'] = np.where(raw_data['인가미인가'] == '인가', raw_data["총분납잔금"], raw_data["현재원금"] + raw_data["현재법비용"])
# raw_data['반영OPB'] = np.where(raw_data['인가미인가'] == '인가', raw_data["총분납잔금"]-raw_data["초과입금"] , raw_data["현재원금"] + raw_data["현재법비용"])


In [122]:
전체요약 = raw_data.query('분납상태=="진행"').groupby(['인가미인가','현재결과']).agg({'분납키':'count', '반영OPB':'sum'})
전체요약.loc[("합계", ""), :] = 전체요약.sum()
전체요약.applymap('{:,.0f}'.format)

분납키          반영OPB
인가미인가 현재결과                          
면책    면책허가결정        2     24,853,989
미인가   개시           86    576,539,347
      금지           99    828,385,786
      폐지취소결정송달      3     34,167,388
      항고            2     25,764,601
인가    인가        3,585  6,461,167,144
      종결            3      4,696,785
      폐지취소결정        6      6,758,634
      폐지취소결정송달     72     91,341,352
      항고           13     35,570,319
폐지    절차폐지결정일       8     51,266,923
      폐지결정          2     21,990,537
합계              3,881  8,162,502,805

### 진행건 분리 - 종결건 제외하지 않아야 함

In [123]:
# 불필요한 열 삭제 - 이미 삭제한 열이 있어도 오류나지 않는 방식
# 삭제할 열 목록
columns_to_drop = ['상태및담당자']
# 남길 열 목록을 원래의 순서대로 계산
columns_to_keep = [col for col in raw_data.columns if col not in columns_to_drop] # difference 함수 쓰면 원래 순서가 엉망으로 바뀌어버림
# 필요한 열만 남기기
raw_data = raw_data[columns_to_keep]

# 진행중, 차주, 보증인 시트 작성(종결인데 분납중지 안 된 것 여기선 제외x)
ing_df = raw_data.query('분납상태 == "진행" and 인가미인가.str.contains("인가|미인가")')

# 종료건 제외 : 일부 계좌만 종결|매각|환매 될 수 있으므로 여기서 제외하지 않는다. -> 종료건 제외도 계좌별 상태가 확정되고 해야 함
# 어차피 통합에선 종결건은 우선 제외하므로 자산정리에 문제는 없음
# 통합파일에서 계좌별로 상태 확인 후 종료인데 중지 안 된 건 처리
# 최초 작성코드
# if path_wd.find("대성") < 0 : # 솔림
#     ing_df = raw_data[~(raw_data.채무상태.str.match("종결|환매|매각")|raw_data.담당자.str.match("종결|매각(?!예정)|DS|환매"))]
# else : # 대성
#     ing_df = raw_data[~(raw_data.채무상태.str.match("종결|환매|매각")|raw_data.담당자.str.match("종결|매각|환매|파산|회계파산"))]

    
###### 신복과 충돌하는 거 제거하기 #############################################################
if path_wd.find("대성") < 0 : # 솔림
    개회제외 = {"채무자" : [], "보증인" : [] } # 20462830 중복입금건. 신복으로 처리하다가 개인회생 면책됨 231106
else : # 대성
    개회제외 = {"채무자" : [], "보증인" : [] } # 
##############################################################################################
if len(개회제외["채무자"]) > 0 : ing_df = ing_df[~(ing_df['채무자키'].isin(개회제외["채무자"]) & (ing_df['분납자관계'] == '채무자'))]
if len(개회제외["보증인"]) > 0 : ing_df = ing_df[~(ing_df['채무자키'].isin(개회제외["보증인"]) & (ing_df['분납자관계'] == '보증인'))]

# 채무자, 보증인
debt_ing = ing_df[ing_df["분납자관계"]=="채무자"]
grt_ing = ing_df[ing_df["분납자관계"]=="보증인"]

# 저장
with pd.ExcelWriter(join(path_wd, name_dataset+"_기준데이터.xlsx"), engine='openpyxl', mode = 'w') as writer :
    ing_df.to_excel(writer, sheet_name='개인회생(진행)', index=False)
    debt_ing.to_excel(writer, sheet_name='개인회생(진행)_채무자', index=False)
    grt_ing.to_excel(writer, sheet_name='개인회생(진행)_보증인', index=False)
    raw_data.to_excel(writer, sheet_name='개인회생(전체)', index=False)

##### 계좌키 나누고 새로운 파일 저장

In [124]:
# 개인회생 계좌 나누기

# path_wd = r"D:\3.자산\전산 dataset\230430(0501)"
# name_dataset = "개인회생새창_20230502_0850_기준데이터"
# ing_df = pd.read_excel(join(path_wd, name_dataset+".xlsx"), sheet_name="개인회생(진행)", dtype={'채무자키':str, '분납키':str, '계좌키':str})

if len(ing_df.계좌키.apply(len).value_counts()) > 1 : # 계좌키의 길이값이 두 종류 이상이라면(통합계좌 있으면)

    split_ing_df = ing_df.copy()
    split_ing_df["비고"] = ""
    new_rows = []
    for index, row in split_ing_df.iterrows():
        account_keys = row['계좌키'].split(',')
        first_account_key = account_keys.pop(0)
        split_ing_df.loc[index, '계좌키'] = first_account_key # 여러 계좌키를 첫번째 계좌키로 값 수정

        for account_key in account_keys:
            new_row = row.copy() # 일단 모든 칼럼 복사
            new_row['계좌키'] = account_key
                                # 전산 OPB는 최초OPB임
            new_row['반영OPB'],new_row['OPB'], new_row['총분납금'],new_row['총분납입금'],new_row['분납미납금'],new_row['분납잔금'],new_row['총분납잔금'] = 0,0,0,0,0,0,0 # 값 안 가져올 컬림은 빈문자열로 대체
            new_row["분납메모"] = "통합신고 " + first_account_key
            new_rows.append(new_row)
            

    temp = pd.DataFrame(new_rows)

    result = pd.concat([split_ing_df, temp], ignore_index=True)

    # 저장
    writer = pd.ExcelWriter(join(path_wd, name_dataset+"_진행중_계좌키분리_기준데이터.xlsx"))
    result.to_excel(writer, sheet_name="개인회생(진행)", index=False)
    # writer.save() # 이제 save()는 없어진 api 안 쓰면 됨.
    writer.close()


### [DC] CHECK

#### 계좌 분리 후 체크할 것

In [126]:
# 면책건

# cols = ["채무자키", "계좌키", "분납자관계", "분납자성명인", "인가미인가", "현재결과", "분납상태", "분납중단일", "중단사유","현재결과일","면책결정일"]
면책 = raw_data.query('인가미인가=="면책" or 중단사유 in ["면책","납부완료","중도완제"] or 면책결정일!=""').copy()

# 계좌키를 쉼표로 분리하여 여러 개의 행으로 분할
splitted_df = pd.DataFrame(면책['계좌키'].str.split(',').tolist(), index=면책.index).stack() # 스택함수를 쓰면 열이 행으로 분리되고, level_0(원래 인덱스), level_1 의 이차원 인덱스가 생긴다.

# 인덱스 재설정
splitted_df = splitted_df.reset_index(level=1, drop=True).rename('분리계좌키') # 레벨0만 남기고 1은 드랍

# 합치기
면책_행분리 = 면책.merge(splitted_df, how='left', left_index=True, right_index=True)

면책_행분리.drop(['계좌키'],axis=1, inplace=True)
면책_행분리.insert(6,"분리계좌키", 면책_행분리.pop("분리계좌키"))
면책_행분리.rename(columns={"분리계좌키" : "계좌키"}, inplace=True)

# 면책_행분리.to_excel(join(path_wd, "[check]개인회생_면책건_계좌키분리_"+cutoff+".xlsx"), index=False)

In [127]:
# 보증인새창조회
####################################################
wd = r"D:\3.자산\전산 dataset\솔림\240630"
grt_ori = pd.read_excel(join(wd, "보증인새창_20240701_1227.xlsx"), dtype=grt_dtype).fillna("")
####################################################

#### 개인회생과 차주상태

In [128]:
# 면책건
차주면책건 = 면책_행분리.query('분납자관계=="채무자"')
보증인면책건 = 면책_행분리.query('분납자관계=="보증인"')

# 면책종결 관련 단어
면책종결 = r"종결|종료|면책|면탈|완납|완제|매각(?!예정)|(?<!피)환매|DS"

# 조회할 칼럼
cols = ["채무자키", "계좌키", "분납키", "분납자관계", "분납자성명인", "인가미인가","현재결과","분납중단일","중단사유","면책결정일","채무상태","담당자","주민번호인"]

In [129]:
# 차주상태 불일치건(기준일 면책건), 누락채권은 누락이 맞음. 
# 동일주민번호에 따른 채무상태 일치여부는 따로 확인하므로 개인회생데이터만봐도 됨.
check차주면책 = 차주면책건.query('~(채무상태.str.contains(@면책종결) or 담당자.str.contains(@면책종결))')[cols]
if len(check차주면책) > 0 :
    print(f"차주 개회 면책인데 전산 상태및담당자는 그렇지 않은 건 : {len(check차주면책)}건")
    check차주면책.to_excel(join(wd,"[check]최근면책건_차주.xlsx"), index=False)
else :
    print("차주 면책 상태및담당자 이상무")
# 누락건 제외하고는 계좌조회새창의 채무상태를 개인회생(면책)으로.

차주 개회 면책인데 전산 상태및담당자는 그렇지 않은 건 : 2건


#### 개인회생과 보증인상태

In [130]:
grt = grt_ori.copy()

In [138]:
# 상태 '종결' 제외
grt1 = grt.query('~보증인상태.str.match("종결|보증면탈") and ~(담당자.str.contains(@종료))') # 종결건은 빼고 살핀다.

# 언더바 제거
grt1["보증인상태"] = grt1.보증인상태.str.replace("_", "")

# 일단 보증인 상태값에 불일치 값이 없는지 체크 # 채무상태는 아쉽게도 없네
보증상태값중복 = grt1.groupby(by=["주민번호인"]).filter(lambda x: x.보증인상태.nunique() != 1)[["채무자키","채무자주민번호", "계좌키", "보증인키", "보증인성명인", "주민번호인", "보증인상태","담당자"]].sort_values(by=["주민번호인"])
if len(보증상태값중복) : 
    print(f"미종결 채권의 동일 보증인상태가 다른건 : {len(보증상태값중복)}건. 엑셀 확인")
    보증상태값중복.to_excel(join(wd, "[check]보증상태값중복.xlsx"), index=False)
else : 
    print("미종결 채권의 보증인 상태값 불일치 이상무 ---")

미종결 채권의 동일 보증인상태가 다른건 : 11건. 엑셀 확인


In [132]:
# 보증인상태 불일치건

# 개인회생새창에는 보증인 주민번호가 없으므로 보증인 주민번호를 먼저 찾아야함.
merged1 = 보증인면책건[cols].merge(grt[["채무자키", "보증인성명인", "주민번호인"]], left_on=["채무자키", "분납자성명인"], right_on=["채무자키", "보증인성명인"], how='left', \
    suffixes=("","_보증인")).drop(["보증인성명인"], axis=1)

# 인가미인가와 보증인 상태를 비교
merged2 = merged1.merge(grt[["주민번호인", "보증인상태"]], left_on=["주민번호인_보증인"], right_on=["주민번호인"], suffixes=("_채무자","_y"), how='left').drop(["주민번호인_y"], axis=1)

# 보증인 개회면책인데 보증인상태가 종결류가 아닌 것
merged3 = merged2.query('~보증인상태.str.contains("종결|면책|면탈|완납", na=False) and ~담당자.str.contains("종결")')

if len(merged3) > 0 :
    print(f"차주 개회 면책인데 전산 상태및담당자는 그렇지 않은 건 : {len(merged3)}건")
    merged3.to_excel(join(wd, "[check]보증인상태면책수정.xlsx"), index=False)
else : 
    print("보증인 면책 상태및담당자 이상무")

차주 개회 면책인데 전산 상태및담당자는 그렇지 않은 건 : 2건


In [133]:
merged2.query('주민번호인_보증인.isin(@보증상태값중복.주민번호인)')

,채무자키,계좌키,분납키,분납자관계,분납자성명인,인가미인가,현재결과,분납중단일,중단사유,면책결정일,채무상태,담당자,주민번호인_채무자,주민번호인_보증인,보증인상태


In [ ]:
# temp = 미종결인가건.copy()
# # 변제일 == 조회일 현재,잔여,미납회차 수정 ★★★★★????
# # 복잡하게 생각하지 말고 컷오프(말일) 다음날 아침에 조회한 경우만을 생각한다. 
# # 이 경우는 입금된 경우가 거의 없을 것이므로 미납회차 -가 안나올 것이고, 나온다면 컷오프일 문제일 것이다.
# # 한 번 확인해보고, 아니다 싶으면 이건 쓰지 않는다. 다만 현재회차차이가 1이고, 변제일이 조회일과 같은 경우 회차오류 무시용으로 사용한다.
# # 말일에 다운 받은 것은 말일과 공휴일여 겹쳤을 때 미리 들어온 금액 때문에 납입회차,미납회차가 영향받아서 복잡함.
# # 휴일때문에 컷오프 + 1+n일에 조회했더라도 법원회차 역시 (1+n)-1일까지는 반영되었기 때문에 신경쓸 필요없다. 이건 분납잔액과는 상관없는 작업이므로
# # 말일 같은 것도 여기서는 신경쓸 필요 없음.
# # 회차수정오류여부 열 만들기
# temp["회차수정오류여부"] = ""

# def 회차수정(row): #(row, 조회일말일여부)
#     조회날짜 = 조회일[-2:] # "두자리숫자"
#     실변제일 = row['실변제일'] # "" or "두자리숫자" or "말일" 
#     회차수정 = False
    
#     # 조회일 == 변제일 or (조회일==말일 & 변제일==말일)
#     if 실변제일 == 조회날짜 :
#         회차수정 = True
        
#     if 회차수정 : 
#         row['현재회차'] -= 1
#         row['잔여회차'] += 1
#         row['미납회차'] -= 1
#         row["회차수정오류여부"] = "수정"
#     else : 
#         row["회차수정오류여부"] = "미수정"
        
#     if row['현재회차'] < 0 : #잔여는 더해주니 음수 불가
#         row["회차수정오류여부"] = "현재회차음수오류발생" # 있을 수 없는 오류
#         print(f"음수 값 발견: index: {row.name}, 채무자키: {row['채무자키']}, 분납키: {row['분납키']}")
#     elif row['미납회차'] < 0 : #미납회차가 음수나는 경우는 컷오프를 당겨줘야 하는 경우. 
#         row["회차수정오류여부"] = "미납회차음수오류, 컷오프 -1개월 수정바람"
#         print(f"음수 값 발견: index: {row.name}, 채무자키: {row['채무자키']}, 분납키: {row['분납키']}")

#     return row

# # 조회말일여부 = is_last_day_of_month(조회일)
# temp = temp.apply(회차수정, axis=1) # (회차수정, args=(조회말일여부,), axis=1)

# # # raw_data 업데이트 하기 ★★★★★ 사용해야 하는 코드로 판명나면 temp대신 미종결인가건으로 바꾼다.
# # raw_data.update(미종결인가건)

# #변제오류체크용 전산-법원
# temp["현재회차차이"] = temp.현재회차 - temp.현재변제회차

# # 확인하고 싶으면 엑셀출력
# 미종결인가건cols = list(dict.fromkeys(기본조회칼럼 + ["실변제일", "실변제금","현재회차차이","현재회차", "현재변제회차","잔여회차","미납회차", "총분납회차",
#                                           "변제회차1", "변제기간1", "변제회차2", "변제기간2", "변제회차3", "변제기간3", "회차수정오류여부"]))
# temp[미종결인가건cols].to_excel(join(path_wd, "개인회생_미인가종결건_"+cutoff+".xlsx"))